In [55]:
#Meelas_codes/ optimum_codes/ code_messiah/ 
import pandas as pd
import numpy as np 
import re # regex- regular expression
import dateparser
from collections import Counter 
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#Loading our txt file 
#Defining a function for that purpose
def read_file(file):
    '''Reads Whatsapp Txt file into list of strings'''
    x = open(file, 'r', encoding = 'utf-8') #opens txt file into variable x but the variable cant be explored yet
    y = x.read() #it becomes a huge chunk of strings that needs to be separated line by line
    content = y.splitlines() #This converts the chunk of strings into a list of strings
    return content

#Now we use our defined function to read our txt file
chat = read_file(r'C:\Users\Abduljawad\Desktop\Analytics_Projects\Group_chat.txt')
#chat
len(chat)

#Next, we remove variables not useful for our analysis
#removing messages containing'joined using this'
join = [line for line in chat if "joined using this" in line]
#print(join)
#clean out empty lines 
chat = [line.strip() for line in chat]
#print("length of chat is:", len(chat))
#clean out lines containing "joined using this"
chat = [line for line in chat if not "joined using this" in line]
len(chat)


#further removal of empty lines 
clean_chat = [line for line in chat if len(line) > 1]
#print("length of clean_chat is:",len(clean_chat))
#null_chat = [line for line in chat if len(line) < 1]
#len(null_chat)
#null_chat

#drop notifications of participants that left 
left = [line for line in clean_chat if line.endswith("left")]
#left
#clean out 'left' notification lines
clean_chat = [line for line in clean_chat if not line.endswith("left")]
clean_chat
#print(msgs[0 : 10])
#removing other redundant notification lines
clean_chat = [line for line in clean_chat if not "Damola created" in line]
clean_chat = [line for line in clean_chat if not "You were added" in line]


#Using regex package to merge messages that belong together
msgs = [] #message box
pos = 0 #position counter for msgs in the box
"""
Flow:
For every line, see if it matches the expression which is starting with the format "number(s)+slash" eg "21/"
if it does, it is a new line of conversion as they begin with dates, add it to message box
Else, it is a continuation of the previous line and append to msgs, then pop previous line
"""

for line in clean_chat:
    if re.findall("\A\d+[/]", line):
        msgs.append(line)
        pos += 1
    else:
        take = msgs[pos - 1] + ". " + line
        msgs.append(take)
        msgs.pop(pos - 1)
len(msgs)
msgs[0 : 10]

#Extract Date, Time, Name and Message content from our msgs
date = [msgs[i].split(',')[0] for i in range(len(msgs))]
len(date)

time = [msgs[i].split('-')[0].split(',')[1] for i in range(len(msgs))]
len(time)

#print(msgs [0:10])

name = [msgs[i].split(':')[1].split('-')[1] for i in range(len(msgs))]
name

content = []
for i in range(len(msgs)):
    try:
        content.append(msgs[i].split(':')[2])
    except IndexError:
        content.append('Missing Text')
len(content)
content

#Now we use pandas library to merge our lists into a DataFrame named df 
df = pd.DataFrame(list(zip(date, time, name, content)), columns = ['Date', 'Time', 'Name', 'Content'])
df

#drop rows returning content = "Missing Text"
df = df[df["Content"]!='Missing Text']
df.reset_index(inplace=True, drop=True)
df

#Creating Datetime column by merging Date and Time columns
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['DateTime']

#Creating a weekday column 
df['weekday'] = df['DateTime'].apply(lambda x: x.day_name())

#Now, we split out 'Content' column into number of letters and number of words 
df['Letter_Count'] = df['Content'].apply(lambda s: len(s))
df['Word_Count'] = df['Content'].apply(lambda s: len(s.split(' ')))

#Next, we split our Time Column to display hour of the day each msg was sent
df['Hour'] = df['Time'].apply(lambda x : x.split(':')[0])

#We can now take a look at the DataFrame 
df.head()

#Now, we can save our DataFrame df in .csv format
#saving in .csv
df.to_csv("GroupChat.csv")